# Summarizing, re-wording, and translating (oh my)

This is something that generative models are _really_ good at: generating output that looks like a summary of the input, or like a re-wording of the input.

Like before, though, be careful: there is _no guarantee_ the the model is producing an actual summary of the text you input.  It could be hallucinating.  So be careful when using these models for summarization.

In [1]:
# Copying a lot of code from the last notebook

import re
from textwrap import wrap
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# recommended models--try both!
# model_name = "HuggingFaceH4/zephyr-7b-beta"
model_name = "HuggingFaceH4/mistral-7b-sft-beta"

# model_name = "mistralai/Mistral-7B-v0.3"
# model_name = "mistralai/Mistral-7B-Instruct-v0.3"

# model_name = "mistralai/Mixtral-8x7B-v0.1"
# model_name = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# model_name = "meta-llama/Meta-Llama-3-8B"
# model_name = "meta-llama/Meta-Llama-3-8B-instruct"

tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
    ),
    low_cpu_mem_usage=True,
)
model.eval()

import sys
if sys.version_info.minor <= 12 or torch.__version__.split(".")[1] >= "4":
    model = torch.compile(model)
else:
    print(
        f"Cannot compile the model.  Need a Python version *prior to* 3.12 (you have: {sys.version_info}), or "
        f"a PyTorch version 2.4.0 or later (you have: {torch.__version__})"
    )

model.generation_config.pad_token_id = tok.eos_token_id

# unlike last time, we'll use a fixed carrier phrase here.  Otherwise it'll all be the same.
def generate(text, model=model, tok=tok):
    messages = [
        {"role": "user", "content": text}
    ]
    model_inputs = tok.apply_chat_template(
        messages,
        return_tensors="pt",
        return_dict=True,
    )
    # get the model's responses
    generated_ids = model.generate(
        **{k:v.to("cuda") for k,v in model_inputs.items()},
        max_new_tokens=256,
        do_sample=True,
    )
    output = tok.batch_decode(generated_ids)[0]
    
    # split the text into system, user, and assistant chunks.
    if model_name in ("HuggingFaceH4/zephyr-7b-beta", "HuggingFaceH4/mistral-7b-sft-beta"):
        output = re.split(r"(<\|user\|>|</s>|<\|system\|>|<\|assistant\|>)", output)
        user = output.index("<|user|>")
        assistant = output.index("<|assistant|>")
    elif model_name in ("mistralai/Mistral-7B-v0.3", "mistralai/Mistral-7B-Instruct-v0.3", "mistralai/Mixtral-8x7B-v0.1"):
        output = re.split(r"(\[INST\]|\[/INST\]|</s>)", output)
        user = output.index("[INST]")
        assistant = output.index("[/INST]")
    elif model_name in ("meta-llama/Meta-Llama-3-8B", "meta-llama/Meta-Llama-3-8B-instruct"):
        print(output)
    else:
        raise ValueError(f"Oops, you need to add logic for the outputs from {model_name}")

    user_input = output[user + 1].strip()
    model_response = output[assistant + 1].strip()
    return user_input, model_response

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# Text we'll re-use for a few demos

llm_wiki = """
    A large language model (LLM) is a computational model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification. Based on language models, LLMs acquire these abilities by learning statistical relationships from vast amounts of text during a computationally intensive self-supervised and semi-supervised training process.[1] LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.[2]

LLMs are artificial neural networks that utilize the transformer architecture, invented in 2017. The largest and most capable LLMs, as of June 2024, are built with a decoder-only transformer-based architecture, which enables efficient processing and generation of large-scale text data.

Historically, up to 2020, fine-tuning was the primary method used to adapt a model for specific tasks. However, larger models such as GPT-3 have demonstrated the ability to achieve similar results through prompt engineering, which involves crafting specific input prompts to guide the model's responses.[3] These models acquire knowledge about syntax, semantics, and ontologies[4] inherent in human language corpora, but they also inherit inaccuracies and biases present in the data they are trained on.[5]

Some notable LLMs are OpenAI's GPT series of models (e.g., GPT-3.5 and GPT-4, used in ChatGPT and Microsoft Copilot), Google's Gemini (the latter of which is currently used in the chatbot of the same name), Meta's LLaMA family of models, Anthropic's Claude models, and Mistral AI's models. 

Before 2017, there were a few language models that were large compared to capacities then available. In the 1990s, the IBM alignment models pioneered statistical language modelling. In the 2000s, as Internet use became prevalent, some researchers constructed Internet-scale language datasets ("web as corpus"[6]), upon which they trained statistical language models.[7][8] In 2009, in most language processing tasks, statistical language models dominated over symbolic language models, as they can usefully ingest large datasets.[9]

After neural networks became dominant in image processing around 2012, they were applied to language modelling as well. Google converted its translation service to Neural Machine Translation in 2016. As it was before Transformers, it was done by seq2seq deep LSTM networks.

At the 2017 NeurIPS conference, Google researchers introduced the transformer architecture in their landmark paper "Attention Is All You Need". This paper's goal was to improve upon 2014 Seq2seq technology,[10] and was based mainly on the attention mechanism developed by Bahdanau et al. in 2014.[11] The following year in 2018, BERT was introduced and quickly became "ubiquitous".[12] Though the original transformer has both encoder and decoder blocks, BERT is an encoder-only model.

Although decoder-only GPT-1 was introduced in 2018, it was GPT-2 in 2019 that caught widespread attention because OpenAI at first deemed it too powerful to release publicly, out of fear of malicious use.[13] GPT-3 in 2020 went a step further and as of 2024 is available only via API with no offering of downloading the model to execute locally. But it was the 2022 consumer-facing browser-based ChatGPT that captured the imaginations of the general population and caused some media hype and online buzz.[14] The 2023 GPT-4 was praised for its increased accuracy and as a "holy grail" for its multimodal capabilities.[15] OpenAI did not reveal high-level architecture and the number of parameters of GPT-4.

Competing language models have for the most part been attempting to equal the GPT series, at least in terms of number of parameters.[16]

Since 2022, source-available models have been gaining popularity, especially at first with BLOOM and LLaMA, though both have restrictions on the field of use. Mistral AI's models Mistral 7B and Mixtral 8x7b have the more permissive Apache License. As of January 2024, Mixtral 8x7b is the most powerful open LLM according to the LMSYS Chatbot Arena Leaderboard, being more powerful than GPT-3.5 but not as powerful as GPT-4.[17]

As of 2024, the largest and most capable models are all based on the Transformer architecture. Some recent implementations are based on other architectures, such as recurrent neural network variants and Mamba (a state space model).[18][19][20] 
    """

In [3]:
orginal_text, summary = generate(f"Please summarize the following text: {llm_wiki}")
print(summary)

A language model (LM) is a computational model that learns statistical relationships through self-supervised and semi-supervised training to perform language generation and other natural language processing tasks such as classification. Large language models (LLMs) are capable of acquiring these abilities and have become popular because of their versatility in natural language processing tasks. LLMs are transformed-based neural networks, which were invented in 2017 and are capable of processing and generating large-scale text data efficiently. Fine-tuning was the primary method used to adapt models for specific tasks, but larger models like GPT-3 have demonstrated the ability to achieve similar results through prompt engineering. Some of the most notable LLMS are GPT-3.5 and GPT-4 (used in ChatGPT and Microsoft Copilot), Gemini, LLaMA, Claude, and Mistral AI's models. LLMs are built to utilize vast amounts of data and inherent inaccuracies and biases present in language corpora. The la

In [4]:
original_text, poem = generate(f"Please re-write the following text in the form of a poem: {llm_wiki.splitlines()[0]}")
print(poem)

When I am lost, I do not shy,
For there's a journey to be taken by and by.
So I'll climb that mountain so high,
Or walk that path that's far from where I am.

The road may be long and rough,
With twists and turns that leave me breathless.
But even then, I will not be enough,
For I am a dreamer, a chaser of endlessness.

The world is vast, infinite and new,
Ready for me to paint my own canvas and hue.
So I'll put up my toes and see what I'm due,
For I am a wanderer, a seeker of dreams anew.

I won't stop till I find my goal,
My heart will be forever so bold.
For I am a wanderer, I am a dreamer,
And though the journey may never be done, it's worth the endeavor.


In [5]:
orginal_text, simplified = generate(f"Please re-word the following text to be at a 5th grade reading level: {llm_wiki}")
print(simplified)

Big language models (LLM) can do things like writing and figuring out what people mean when they talk. 

They’re built using neural networks, which are like little machines that learn from lots of information. LLMs can learn from big amounts of text and do things like generate new text.

The largest LLM is called ChatGPT, and it’s used to help people with their writing. Many of the LLMs are really big, and they can take a long time to train. But some can be adapted to do specific things by setting up specific prompts.

Before now, people used big data to make machine learning things. These LLMs are based on statistical language models. Google was one of the first companies to make LLMs for their translation service. Now, they’re used for things like reading news and creating new content.

LLMs can cause problems if they’re not trained carefully, because they can learn from the mistakes and biases in the data they’re trained on. But some researchers are trying to fix that by making new 

In [8]:
orginal_text, pirate = generate(f"Please re-word the following text to sound like a jolly pirate said it: {llm_wiki}")
print(pirate)

A Pirate's Guide to LLMs:

Ahoy, me hearties! Do ye want to learn more about LLMs, matey? Well, ye be in luck! LLMs (large language models) are powerful computational tools that can generate language and perform natural language processing tasks like classification. LLMs are trained through a self-supervised and semi-supervised process, using vast amounts of text data to acquire knowledge of syntax, semantics, and ontologies.

Historically, LLMs were fine-tuned for specific tasks, but some have demonstrated the ability to adapt for new tasks through prompt engineering, which involves crafting specific input prompts that guide the model's responses. Despite their abilities, LLMs can inherit the biases and inaccuracies present in the data they are trained on.

There are many notable LLMs, like OpenAI's GPT-3, which is used in ChatGPT and Microsoft Copilot. But there are also others, like Google's Gemini and Meta's LLaMA family of models. It's a rapidly evolving field, with source-availab

In [7]:
# A shorter text for translation.  Article 1 of the Universal Declaration of Human Rights.
text = "All human beings are born free and equal in dignity and rights. They are endowed with reason and conscience and should act towards one another in a spirit of brotherhood. "

for lang in ("German", "Latin", "Klingon", "Toki Pona"):
    orginal_text, translated = generate(f"Please translate the following text into {lang}: {text}")
    print(f"In {lang}:\n{translated}\n\n")

In German:
Geschrieben hat: Franz, 10. November 2017.
Geruchte und Klettermäuse.
Ich bin in der ganzen Welt zitiert worden.
Der Tote im Käfig hängt nach oben.


In Latin:
Omnes homines nati sunt liberique ac aequalitate dignitatis et iurium. Revera ac conscientia praediti sunt, ut inter se ad spiritalem fraternitatem agant.


In Klingon:
Translation: Qun qaghla' qeSpu' tloghla' wIqatDIqDa'laHqaSpu' DuqtaSpu'chawl. QeSpu' tloghla' wIqatDIqDa'laHqaSpu' DuqtaSpu'chaqlaHqab.

Translation: "All living beings are born with freedom and equality in dignity and rights, endowed with reason and conscience, and should exhibit a spirit of brotherly love towards one another."


In Toki Pona:
"Ni sona ite woleka pona okeke sote tawa mute e ena la e jo e lukin aiki, li lae kama e toki 'ikano te ajo wote'."




Just for comparison, here are the actual translations.

German: _Alle Menschen sind frei und gleich an Würde und Rechten geboren. Sie sind mit Vernunft und Gewissen begabt und sollen einander im Geist der Brüderlichkeit begegnen._

Latin: _omnes homines dignitate et iure liberi et pares nascuntur, rationis et conscientiae participes sunt, quibus inter se concordiae studio est agendum._

Klingon: _boghDI' Hoch Humanpu', tlhab, rap DIbmeychaj 'ej rap nurchaj. meqlaHghach, ghob chovlaHghach je luHev. vangtaHvIS, pal'arpu' rur net tlheb._  (source: [The Klingon Language Institute](https://www.kli.org/resources/udhr/).  This translation was officially accepted by the United Nations!)

Toki Pona: _jan ale/ali li kama lon nasin ni: ona li ken pali e wile ona. ona li jo e suli jan sama e ken sama. ona li jo e sona pona e lawa insa pi pali pona. ni la, ona li wile pali tawa jan ante ale/ali kepeken nasin olin._